In [ ]:
import re
from collections import defaultdict
from datetime import datetime, timedelta
from math import nan, pi
from typing import cast

from matplotlib import pyplot as plt

In [ ]:
RPM_TO_MPS = 1 / 60 * 2 * pi * 0.85 / 24.67

SAVE_DATA_PATTERN = re.compile(
    r"""(?x)
        Save\ Data\ AGV\ CENTERX:(?P<x_center>-?\d+\.\d+),
        CENTERY:(?P<y_center>-?\d+\.\d+),
        heading:(?P<heading>-?\d+\.\d+),
        valid:(?P<valid>\d),
        mode:(?P<mode>\d)
    """
)

data: defaultdict[str, list[float | datetime]] = defaultdict(list)
for file_path in [
    "tracking_control_node.wisdom-OptiPlex-9020.root.log.INFO.20260113-040502.27022",
    # "tracking_control_node.wisdom-OptiPlex-9020.root.log.INFO.20260113-041911.27022",
    # "tracking_control_node.wisdom-OptiPlex-9020.root.log.INFO.20260113-042001.27022",
    # "tracking_control_node.wisdom-OptiPlex-9020.root.log.INFO.20260113-083737.27022",
    # "tracking_control_node.log.1",
    # "tracking_control_node.log",
]:
    with open(file_path, "r") as log_file:
        for line in log_file:
            # prefix, sep, line = line.partition(" - ")
            # if not sep:
            #     continue

            # prefix, sep, line = line.rpartition("]: ")
            # if not sep:
            #     continue
            # _prefix, sep, timestamp_str = prefix.rpartition("] [")
            # if not sep:
            #     continue

            if not line.startswith("tracking_control_node ["):
                continue
            prefix, sep, line = line.partition(" [")
            if not sep:
                continue
            timestamp_str, sep, line = line.partition(":")
            if not sep:
                continue
            line = line.strip()
            if line[-1] == "]":
                line = line[:-1]

            if line.startswith("CyclicTime: "):
                if "cyclic_time" not in data:
                    data.clear()
                data["cyclic_time"].append(float(line.split(": ")[-1]))

                # timestamp = datetime.strptime(prefix[:19], "%Y-%m-%d %H:%M:%S")
                timestamp = datetime.fromtimestamp(float(timestamp_str))
                if len(data["timestamp"]):
                    last_timestamp = cast(datetime, data["timestamp"][-1])
                    if last_timestamp >= timestamp:
                        cyclic_time = timedelta(
                            seconds=cast(float, data["cyclic_time"][-1])
                        )
                        timestamp = last_timestamp + cyclic_time
                data["timestamp"].append(timestamp)

                for key in data.keys():
                    if key == "cyclic_time":
                        continue
                    while len(data[key]) < len(data["cyclic_time"]):
                        data[key].append(nan)

            if match := SAVE_DATA_PATTERN.search(line):
                data["x_center"].append(float(match.group("x_center")))
                data["y_center"].append(float(match.group("y_center")))
                data["heading"].append(float(match.group("heading")))
                data["valid"].append(int(match.group("valid")))
                data["mode"].append(int(match.group("mode")))

            if "gFrontRoute.PathIndex_offsetAngle = " in line:
                data["front_index_angle"].append(int(line.split(" = ")[-1]))

            if "gFrontRoute.PathIndex_offsetDistance = " in line:
                data["front_index_dis"].append(int(line.split(" = ")[-1]))

            if "gRearRoute.PathIndex_offsetAngle = " in line:
                data["rear_index_angle"].append(int(line.split(" = ")[-1]))

            if "gRearRoute.PathIndex_offsetDistance = " in line:
                data["rear_index_dis"].append(int(line.split(" = ")[-1]))

            if "AGV_MotionStateData.Speed_Global.Vs --> " in line:
                data["v_gs"].append(float(line.split(" --> ")[-1]))

            if "2 AGV_MotionStateData.RunningState --> " in line:
                data["running_state"].append(int(line.split(" --> ")[-1]))

            if "PathOffset_FrontAnn.OffsetX_ToEndPoint = " in line:
                data["offset_x_front"].append(float(line.split(" = ")[-1]))

            if "PathOffset_FrontAnn.OffsetY_ToTargetNow = " in line:
                data["offset_y_front"].append(float(line.split(" = ")[-1]))

            if "PathOffset_RearAnn.OffsetY_ToTargetNow = " in line:
                data["offset_y_rear"].append(float(line.split(" = ")[-1]))

            if "MotionControlData.Command.SteerAngle_FS = " in line:
                data["cmd_angle_fs"].append(float(line.split(" = ")[-1]))

            if "MotionControlData.Command.SteerAngle_RS = " in line:
                data["cmd_angle_rs"].append(float(line.split(" = ")[-1]))

            if "MotionControlData.Command.Speed_Front = " in line:
                data["cmd_speed_f"].append(float(line.split(" = ")[-1]) * RPM_TO_MPS)

            if "MotionControlData.Command.Speed_Rear = " in line:
                data["cmd_speed_r"].append(float(line.split(" = ")[-1]) * RPM_TO_MPS)

data_size = min(map(len, data.values()))
for key in data.keys():
    assert len(data[key]) >= data_size
    data[key] = data[key][:data_size]
raw_data = data

print(f"Loaded {data_size} row(s) of data.")


In [ ]:
data = raw_data.copy()
for key in data.keys():
    data[key] = data[key][int(data_size * 0.0) : int(data_size * 1.0)]
print(data["timestamp"][0])
print(data["timestamp"][-1])

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    cast(list, data["timestamp"]),
    cast(list[float], data["cyclic_time"]),
    ".-",
    label="cyclic_time",
)
plt.xlim((data["timestamp"][0], data["timestamp"][-1]))
plt.legend()
plt.grid(alpha=0.2)
pass

In [ ]:
plt.figure(figsize=(12, 6))
x = cast(list[float], data["x_center"])
y = cast(list[float], data["y_center"])
plt.plot(x, y, "+-", label="center")
plt.xlim(min(x) - 1, max(x) + 1)
plt.ylim(min(y) - 1, max(y) + 1)
plt.legend()
plt.grid(alpha=0.2)
plt.gca().set_aspect("equal")
pass

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    cast(list, data["timestamp"]),
    cast(list[float], data["x_center"]),
    ".-",
    label="x_center",
)
plt.xlim((data["timestamp"][0], data["timestamp"][-1]))
# plt.ylim(1739.2, 1739.3)
plt.legend()
plt.grid(alpha=0.2)
pass

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    cast(list, data["timestamp"]),
    cast(list[float], data["y_center"]),
    ".-",
    label="y_center",
)
plt.xlim((data["timestamp"][0], data["timestamp"][-1]))
# plt.ylim(205.9, 206)
plt.legend()
plt.grid(alpha=0.2)
pass

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    cast(list, data["timestamp"]),
    [
        # ((heading if heading <= 0.5 * pi else (heading - 2 * pi)) * 180 / pi)
        heading * 180 / pi
        for heading in cast(list[float], data["heading"])
    ],
    ".-",
    label="heading",
)
plt.xlim((data["timestamp"][0], data["timestamp"][-1]))
# plt.ylim(268, 272)
plt.legend()
plt.grid(alpha=0.2)
pass

In [ ]:
if "running_state" in data:
    plt.figure(figsize=(12, 4))
    plt.plot(
        cast(list, data["timestamp"]),
        cast(list[float], data["running_state"]),
        ".-",
        label="running_state",
    )
    plt.xlim((data["timestamp"][0], data["timestamp"][-1]))
    plt.legend()
    plt.grid(alpha=0.2)
    pass

In [ ]:
plt.figure(figsize=(12, 6))
for label, color in [
    ("front_index_angle", "r"),
    ("front_index_dis", "orange"),
    ("rear_index_angle", "b"),
    ("rear_index_dis", "skyblue"),
]:
    plt.plot(
        cast(list, data["timestamp"]),
        cast(list, data[label]),
        ".-",
        color=color,
        label=label,
    )
plt.xlim((data["timestamp"][0], data["timestamp"][-1]))
plt.legend()
plt.grid(alpha=0.2)
pass

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    cast(list, data["timestamp"]),
    cast(list[float], data["cmd_speed_f"]),
    "c.-",
    label="cmd_speed_f",
)
plt.plot(
    cast(list, data["timestamp"]),
    cast(list[float], data["cmd_speed_r"]),
    "m.-",
    label="cmd_speed_r",
)
plt.plot(
    cast(list, data["timestamp"]),
    [v for v in cast(list[float], data["v_gs"])],
    "k.-",
    label="v_gs",
)
plt.axhline(0, ls="--", c="gray", alpha=0.3)
plt.xlim((data["timestamp"][0], data["timestamp"][-1]))
plt.legend()
plt.grid(alpha=0.2)
pass

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    cast(list, data["timestamp"]),
    cast(list[float], data["cmd_angle_fs"]),
    "c.-",
    label="cmd_angle_fs",
)
plt.plot(
    cast(list, data["timestamp"]),
    cast(list[float], data["cmd_angle_rs"]),
    "m.-",
    label="cmd_angle_rs",
)
plt.axhline(0, ls="--", c="gray", alpha=0.3)
plt.xlim((data["timestamp"][0], data["timestamp"][-1]))
plt.legend()
plt.grid(alpha=0.2)
pass


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    cast(list, data["timestamp"]),
    cast(list[float], data["offset_x_front"]),
    ".-",
    label="offset_x_front",
)
plt.axhline(0, ls="--", c="gray", alpha=0.3)
plt.xlim((data["timestamp"][0], data["timestamp"][-1]))
# plt.ylim(0, 0.3)
plt.legend()
plt.grid(alpha=0.2)
pass

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    cast(list, data["timestamp"]),
    cast(list[float], data["offset_y_front"]),
    "c.-",
    label="offset_y_front",
)
plt.plot(
    cast(list, data["timestamp"]),
    cast(list[float], data["offset_y_rear"]),
    "m.-",
    label="offset_y_rear",
)
plt.axhline(0, ls="--", c="gray", alpha=0.3)
plt.xlim((data["timestamp"][0], data["timestamp"][-1]))
plt.legend()
plt.grid(alpha=0.2)
pass